In [1]:
import cv2 as cv
import mediapipe as mp
import time
import math
import numpy as np

In [2]:
def putcolor(img):
    cv.rectangle(img,(0,0)             ,(int(w/4.0),int(h/8.0))    ,(255,0,0)    ,-1)#blue
    cv.rectangle(img,(int(w/4.0),0)    ,(int(w/2.0),int(h/8.0))    ,(0,255,0)    ,-1)#green
    cv.rectangle(img,(int(w/2.0),0)    ,(int((3*w)/4.0),int(h/8.0)),(0,0,255)    ,-1)#red
    cv.rectangle(img,(int((3*w)/4.0),0),(w,int(h/8.0))             ,(255,255,255),-1)#white
    cv.rectangle(img,(0,int(h/8.0)+1),(int(h/8.0),int((5*h)/8.0))  ,(255,255,255),-1)#whitesize
    cv.rectangle(img,(0,int(h/8.0)+1),(int(h/8.0),int(h/4.0))      ,(0,0,0),2)
    cv.rectangle(img,(0,int(h/4.0)),(int(h/8.0),int((3*h)/8.0))    ,(0,0,0),2)
    cv.rectangle(img,(0,int((3*h)/8.0)),(int(h/8.0),int(h/2.0))    ,(0,0,0),2)
    cv.rectangle(img,(0,int(h/2.0)),(int(h/8.0),int((5*h)/8.0))    ,(0,0,0),2)
    cv.putText(img, "10",(25, 150), cv.FONT_HERSHEY_PLAIN,2,(0,0,0),3)
    cv.putText(img, "30",(25, 240), cv.FONT_HERSHEY_PLAIN,2,(0,0,0),3)
    cv.putText(img, "50",(25, 330), cv.FONT_HERSHEY_PLAIN,2,(0,0,0),3)
    cv.putText(img, "clear all",(10, 410), cv.FONT_HERSHEY_PLAIN, 1,(0,0,0),1)
    return img

In [3]:
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.75)

I0000 00:00:1718863136.494789 2175379 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1


In [4]:
cap = cv.VideoCapture(0)
_ , frame= cap.read()
x,y,z=frame.shape
t0=0
xs,ys=0,0
blank_image=np.zeros((x, y, z), np.uint8)
brushcolor=(255,0,0)
size=10
while True:
    success, img = cap.read()
    img=cv.resize(img,(1280, 720))
    img=cv.flip(img,1)
    imgRGB=cv.cvtColor(img, cv.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    h,w,c = img.shape
    #h720,w1280
    img=putcolor(img)
    if results.multi_hand_landmarks: 
        for hand_landmarks in results.multi_hand_landmarks:
            x4,y4=(int(hand_landmarks.landmark[4].x*w),int(hand_landmarks.landmark[4].y*h))
            x8,y8=(int(hand_landmarks.landmark[8].x*w),int(hand_landmarks.landmark[8].y*h))
            dist=math.hypot(x4-x8,y4-y8)
            if(dist<75):
                xs,ys=0,0
                cv.circle(img,(int((x4+x8)/2),int((y4+y8)/2)),size+2,(255,255,255),cv.FILLED)
                cv.circle(img,(int((x4+x8)/2),int((y4+y8)/2)),size,brushcolor,cv.FILLED)
                x,y=int((x4+x8)/2), int((y4+y8)/2)
                if 0<=x<=(int(w/4.0)) and 0<=y<=int(h/8.0):
                    brushcolor=(255,0,0)
                elif int(w/4.0)<=x<=int(w/2.0) and 0<=y<=int(h/8.0):
                    brushcolor=(0,255,0)
                elif int(w/2.0)<=x<=int((3*w)/4.0) and 0<=y<=int(h/8.0):
                    brushcolor=(0,0,255)
                elif int((3*w)/4.0)<=x<=w and 0<=y<=int(h/8.0):
                    brushcolor=(0,0,0)
                elif 0<=x<=int(h/8.0) and int(h/8.0)+1 <= y <= int(h/4.0):
                    size=10
                elif 0<= x <=int(h/8.0) and int(h/4.0) <= y <=int((3*h)/8.0):
                    size=30
                elif 0<= x <=int(h/8.0) and int((3*h)/8.0)<= y <= int(h/2.0):
                    size=50
                elif 0<= x <=int(h/8.0) and int(h/2.0)<= y <=int((5*h)/8.0):
                    blank_image=np.zeros((720,1280, z), np.uint8)
            else:
                if(xs==0 & ys==0):
                    xs,ys=x8,y8
                cv.circle(img,(xs,ys),size,brushcolor,cv.FILLED)
                cv.line(blank_image,(xs,ys),(x8, y8),brushcolor,size,cv.FILLED)
                xs,ys=x8,y8
            cv.line(img,(x8,y8-3),(x8, y8+3),(255,255,255),2,cv.FILLED)
            cv.line(img,(x8-3,y8),(x8+3, y8),(255,255,255),2,cv.FILLED)
    t1=time.time()
    cv.putText(img, str(int(1/(t1 - t0)))+ "fps", (20, 700), cv.FONT_HERSHEY_PLAIN, 2,(255,255,255),3)
    t0=t1
    imgray=cv.cvtColor(blank_image,cv.COLOR_BGR2GRAY)
    _,imgInv = cv.threshold(imgray, 0, 255, cv.THRESH_BINARY_INV)
    imgInv=cv.cvtColor(imgInv,cv.COLOR_GRAY2BGR)
    img=cv.bitwise_and(img,imgInv)
    img=cv.bitwise_or(img,blank_image)
    cv.imshow("Video", img)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv.destroyAllWindows()


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1718863136.518028 2198143 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1718863136.525332 2198149 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/Users/jatinkushwaha/anaconda3/envs/venv-metal/lib/python3.11/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


: 